# Social Computing - Summer 2017
# Exercise 4 - Clustering

## Problem 4.1. K-means Clustering of couse network data

Write a Python program that computes K-means clustering for the course dataset. The input dataset (file: networkinput.csv) contains data from our social networking platform. As always this data has been anonymized. Each line in the file represents one feature vector and is associated with a participant in the social networking platform. The vectors have a name ("userXYZ") and the following 4 features:
* Number of posts
* Number of comments
* Number of likes (on both posts and comments)
* Number of friends and followers (average)

By clustering the participants, you identify particpants with similar activity patterns. This can be helpful for research but also for advertisers and polling firms.

Your program should compute K-means clustering for the dataset according to the formula discussed in the lecture ==> Minimizing the objective function:
$$\sum_{k=1}^{K} \sum_{\{n|x_n \in C_k \}} \|x_n - \mu_k\|^2$$
Such that:
$K$ is the number of clusters, $x_n$ is the nth point that belongs to the $k$th cluster, and $\mu_k$ is the centroid (prototype) of the $k$th cluster. (Refer to the lecture for details)
The K-means clustering algorithm should proceed as follows:
1. The program should start by parsing the dataset 
2. Assign four random centroids (prototypes).
3. Assign data points to the nearest centroid. 
4. Recompute the centroid values: The new centroid values of the kth centroid are calculated as the average values of the points currently in that centroid.
5. Repeat from point 3 till the values of the centroids don't change anymore.

<b>The output of your program should be a a list that asigns a cluster ID (0, 1, 2, 3) to every user in the input file. </b>
The first argument in that tuple should be the users's name (e.g., "user111") and the second argument should be the centroid id to which this user is associated to. e.g ('user111', 3).
**Note:** this output value should be the final centroid values that don't change anymore.


<b> After the clustering with k=4 is complete, run the code with the following centroid starting points:

centroids = {0: [9, 33, 29, 25], 1: [4, 44, 12, 41], 2: [10, 13, 44, 65], 3: [10, 44, 48, 70]}.

Have a look at the result and describe the common properties in each of the four groups</b> (max 5 sentences).


In [1]:
import csv 
import random
import numpy as np 
from scipy.spatial import distance 
from collections import OrderedDict
from __future__ import division

# TODO: Read the network activity data set into a dictionary (observations_dictionary)
def read_data_set(data_set_path):
    reader = csv.reader(open(data_set_path))
    observations_dictionary = OrderedDict()
    for row in reader:
        key = row[0]
        observations_dictionary[key] = row[1:]
        for i in observations_dictionary.keys():
            observations_dictionary[i] = [int(j) for j in observations_dictionary[i]]
    return observations_dictionary

# TODO: Assign random centroids
# Hint: centroid values should be randomly assigned so that for each dimension of the centroid, the random values should fall
# between the maximum and the minimum value of that dimension of all the points in the data set.
# For example: for the dimension "Number of Comments", if the maximum value = 49, and the minimum value = 0, the value assigned to
# the dimension "Murder" of the centroid should be randomly assigned between 0 and 49
def create_random_centroid_values(observations, centroid,k):
    # find min and max of observations in each dimension
    number_of_posts, number_of_comments, number_of_likes, number_of_friends_and_followers = [],[],[],[]
    for row in observations.values():
        number_of_posts.append(int(row[0]))
        number_of_comments.append(int(row[1]))
        number_of_likes.append(int(row[2]))
        number_of_friends_and_followers.append(int(row[3]))
    min_posts = min(number_of_posts)
    max_posts = max(number_of_posts)
    min_comments = min(number_of_comments)
    max_comments = max(number_of_comments)
    min_likes = min(number_of_likes)
    max_likes = max(number_of_likes)
    min_friends = min(number_of_friends_and_followers)
    max_friends = max(number_of_friends_and_followers)
    # create dictionary with four random centroids within the observed space
    for i in range(0,k):
        centroid_list =[]
        centroid_list.append(random.randint(min_posts,max_posts))
        centroid_list.append(random.randint(min_comments,max_comments))
        centroid_list.append(random.randint(min_likes,max_likes))
        centroid_list.append(random.randint(min_friends,max_friends)) 
        centroid[i] = centroid_list
    return centroid  

# create_random_centroid_values(read_data_set('networkinput.csv'),{},4)

# Assign centroid ID for each of the data points. Each data item is assigned to its closest centroid
def update_observation_centroids(observations, centroids, k, observation_centroids = None):
    # create dictionary mapping each observation to a centroid index
    # initial run: For each centroid i of the k centroids, create random values
    observation_centroids = {}
    if centroids == None:
        centroids = create_random_centroid_values(observations, {},k)
                 
    # TODO: Each centroid i of the K centroids is the average of the data values previously assigned to that centroid i                
    for key in observations:
        u_id_values = observations[key]
        center = centroids[0]
        observation_centroids[key] = 0
        smallest_dist = distance.euclidean(u_id_values, centroids[0])
        for i in range(1,k): 
            dist = distance.euclidean(u_id_values, centroids[i])
            if(dist<smallest_dist):
                smallest_dist = dist        
                observation_centroids[key] = i
            # TODO: Return the newly created observation centroids
    return observation_centroids

# update_observation_centroids(read_data_set('networkinput.csv'), None, 4, observation_centroids = None)


# Assign centroid ID for each of the data points. Each data item is assigned to its closest centroid
def update_observation_centroids(observations, centroids, k, observation_centroids = None):
# create dictionary mapping each observation to a centroid index
    observation_centroids = {}
    # initial run: For each centroid i of the k centroids, create random values
    for key in observations:
        x = observations[key]
        c = centroids[0]
        observation_centroids[key] = 0
        best_fit = distance.euclidean(np.array(x),np.array(c))
        for i in range(1,k):
            c = centroids[i]
            distance_i = distance.euclidean(np.array(x),np.array(c))
            if(distance_i<best_fit):
                observation_centroids[key] = i
                best_fit = distance_i       
    # TODO: return the updated centroids
    return observation_centroids


# Create new centroid values for each cluster as the mean of data values for the points in that cluster 
def update_centroid_values(observations, observation_centroids, k):
    c_set = {0:[],1:[],2:[],3:[]}
    new_centroids = {}
    for u_id, c_id in observation_centroids.iteritems():
        c_set.setdefault(c_id,[]).append(u_id)
    for i in range(0,k):
        set_length = len(c_set[i])
        sum_of_posts = 0
        sum_of_comments = 0
        sum_of_likes = 0
        sum_of_friends = 0
        for u_id in range(0,set_length):
            #sum of posts is the posts corresponding to the user_id 
            sum_of_posts = sum_of_posts + observations[c_set[i][u_id]][0] 
            sum_of_comments = sum_of_comments + observations[c_set[i][u_id]][1]
            sum_of_likes = sum_of_likes + observations[c_set[i][u_id]][2]
            sum_of_friends = sum_of_friends + observations[c_set[i][u_id]][3]
        if(set_length == 0):
            set_length = 1        
        new_centroids[i] = [(sum_of_posts*1.0)/set_length, (sum_of_comments*1.0)/set_length, (sum_of_likes*1.0)/set_length, (sum_of_friends*1.0)/set_length]
    return new_centroids   

            
def calculate_k_means_clustering(data_set_path, k):
    observations = read_data_set(data_set_path)
#     centroids = update_observation_centroids(observations, k)
    centroids = None
    new_centroids = {0: [9, 33, 29, 25], 1: [4, 44, 12, 41], 2: [10, 13, 44, 65], 3: [10, 44, 48, 70]}
    observation_centroids = None
    # TODO: Compare the new centroid dictionary with the old centroid dictionary
    while (new_centroids != centroids):# new and old centroid dictionaries are NOT similar, do:
        centroids = new_centroids
        observation_centroids = update_observation_centroids(observations, centroids, k, observation_centroids)
        new_centroids = update_centroid_values(observations, observation_centroids, k)        
    return new_centroids,observation_centroids
        
        
        
# run code
calculate_k_means_clustering('networkinput.csv', 4)

# # print update_centroid_values(observations, 4,{0: [9, 33, 29, 25], 1: [4, 44, 12, 41], 2: [10, 13, 44, 65], 3: [10, 44, 48, 70]})

({0: [0.8426966292134831,
   7.348314606741573,
   6.719101123595506,
   9.52808988764045],
  1: [0.05813953488372093,
   0.7209302325581395,
   0.36046511627906974,
   2.546511627906977],
  2: [2.96, 11.44, 25.56, 13.76],
  3: [5.333333333333333, 29.333333333333332, 55.0, 55.333333333333336]},
 {'user1003': 2,
  'user1012': 0,
  'user10132': 1,
  'user10138': 1,
  'user10144': 1,
  'user10159': 1,
  'user10168': 1,
  'user1021': 1,
  'user10258': 1,
  'user10465': 1,
  'user10474': 1,
  'user1066': 0,
  'user11137': 1,
  'user1126': 0,
  'user11266': 1,
  'user1129': 2,
  'user11494': 1,
  'user115': 0,
  'user11509': 1,
  'user11563': 1,
  'user11584': 1,
  'user11587': 1,
  'user11602': 1,
  'user11608': 1,
  'user11629': 1,
  'user11632': 1,
  'user11638': 1,
  'user11644': 1,
  'user11647': 1,
  'user11650': 1,
  'user11653': 1,
  'user11656': 1,
  'user11659': 1,
  'user11662': 1,
  'user11665': 1,
  'user11674': 1,
  'user11677': 1,
  'user11683': 1,
  'user11692': 1,
  'user116

After the clustering with k=4 is complete, with the centroid starting points:
centroids = {0: [9, 33, 29, 25], 1: [4, 44, 12, 41], 2: [10, 13, 44, 65], 3: [10, 44, 48, 70]}, the K means algorithm took 10 rounds to converge.The centroids are the following:
 {0: [0.8426966292134831, 7.348314606741573, 6.719101123595506, 9.52808988764045], 1: [0.05813953488372093,  0.7209302325581395, 0.36046511627906974, 2.546511627906977], 2: [2.96, 11.44, 25.56, 13.76],
  3: [5.333333333333333, 29.333333333333332, 55.0, 55.333333333333336]}
 
89 users in cluster_0, 258 users in cluster_1, 25 users in cluster_2 and 3 users in cluster_4.
Most of the users are in cluster 1 near the centroid [0.05813953488372093,  0.7209302325581395, 0.36046511627906974, 2.546511627906977] which are the lowest centroid values.

In all the 4 clusters, the dimension for the number of posts is the minimum as compared to other dimensions.


 


## Optional: Problem 4.2. Girvan-Newman Algorithm 
The Girvan-Newman algorithm is an efficient algorithm for computing graph clustering.

Write a Python program that computes the best clustering for the Krackhardt Kite graph.
Remember from the lecture that the central idea of the Girvan-Newman algorithm is to compute the edge betweenness in the input graph. Large betweenness value is an indication that the corresponding edge is a bridge between two clusters in the graph, and cutting that edge means isolating those clusters.
The algorithm proceeds by determining edge betweenness values for all the edges in the graph, and removing the edge with the highest betweenness value and repeating until there are no more edges.
The output of the Girvan-Newman algorithm is a dendrogram of clusters where individual vertices are at the bottom. Therefore, it's necessary to cut that dendrogram and determine the best cluster. Best clustering is the one with the highest graph modularity value, which is determined by the formula:
$$ Q = \sum_{i} (e_{ii} - a_i^2) $$
where $e_{ii}$ sums the fraction of graph edges that connects nodes in the ith cluster. And $a_i$ is the fraction of edges that connect to the ith cluster (see lecture for details)
The input to the Python program will be the Krackhardt Kite Graph as an igraph Graph object. The output should be a tuple of two arguments:
The first should be the value of the best modularity corresponding to the best graph clustering (a floating point number). The second argument should be a tuple of igraph Graph objects representing the clusters. 
HINT: The edge betweenness calculation is a variation of the Ulrik Brandes algorithm for calculating betweenness centrality (exercise 1). The attached paper explains how to modify the algorithm to calculate edge betweenness.

In [5]:
# import igraph
# import Queue
# from __future__ import division
# import numpy as np


# def calculate_node_degrees(g):
#     # HINT: You will need to calculate the adjacency matrix of the graph
#     # Some Graph methods from igraph could be helpful

# def calculate_modularity(g, original_deg_dict, m):
#     modularity = 0
#     degree_dict = calculate_node_degrees(g)
#     connected_components = g.components()
#     for i in range(len(connected_components)):
#         subgraph = connected_components.subgraph(i) # each subgraph represents a cluster in the graph
#         e = 0 # Fraction of edges that connect to cluster
#         a = 0 # Fraction of edges that connect to cluster with random connections between edges
#         for v in subgraph.vs:
#             e += degree_dict[v.index]
#             a += original_deg_dict[v.index]
#         # TODO: Calculate the modularity
#     return modularity
        
# # TODO: Calculate edge betweeness  
# def calculate_edge_betweenness(g):

# def calculate_girvan_newman_clustering(g):
#     m = # TODO: The original number of edges
#     original_degree = calculate_node_degrees(g)
#     while # TODO: Graph still has edges:
#         Q = calculate_modularity(g, original_degree, m)
#         if (Q > largest_modularity):
#             largest_modulatirty = Q
#             edge_betweenness = calculate_edge_betweenness(g) # TODO: get edge with the highest betweenness value
#             # TODO: delete the edge with the maximum betweenness value
#             # Hint: Some built-in methods from igraph could be helpful
    
#     # Finally
#     return final_graph_clustering, largest_modularity

# # Program's entry point:
# g = igraph.Graph.Famous('Krackhardt_Kite') # Connected, Unweighted, undirected social network
# calculate_girvan_newman_clustering(g)